<a href="https://colab.research.google.com/github/coder-penguin/ML/blob/master/tutorial/frugally_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sample code for Keras to C++ conversion with frugally-deep

Reference:
https://github.com/Dobiasd/frugally-deep

###Create a sample model

In [0]:
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [0]:
#check the current format
K.image_data_format()

'channels_last'

In [0]:
#set channels_last format if not
K.set_image_data_format('channels_last')

In [0]:
inputs = Input(shape=(4,))
x = Dense(5, activation='relu')(inputs)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [0]:
model.fit(
    np.asarray([[1, 2, 3, 4], [2, 3, 4, 5]]),
    np.asarray([[1, 0, 0], [0, 0, 1]]), epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 2.5915
Epoch 2/10
1/1 [==============================] - 0s 2ms/step - loss: 2.5666
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 2.5483
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 2.5313
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 2.5145
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 2.4974
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 2.4799
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 2.4621
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 2.4439
Epoch 10/10
1/1 [==============================] - 0s 1ms/step - loss: 2.4255


In [0]:
model.save('keras_model.h5', include_optimizer=False)

###Convert the model to a frugally-deep format

In [0]:
!git clone https://github.com/Dobiasd/frugally-deep.git

Cloning into 'frugally-deep'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 8122 (delta 129), reused 153 (delta 80), pack-reused 7903
Receiving objects: 100% (8122/8122), 3.00 MiB | 11.69 MiB/s, done.
Resolving deltas: 100% (5699/5699), done.


In [0]:
!ls ./

frugally-deep  keras_model.h5  sample_data


In [0]:
!python3 ./frugally-deep/keras_export/convert_model.py keras_model.h5 fdeep_model.json

loading keras_model.h5
2020-04-14 01:31:21.133627: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Forward pass took 0.145329 s.
Forward pass took 0.024316 s.
Forward pass took 0.028699 s.
Starting performance measurements.
Forward pass took 0.02438 s.
Forward pass took 0.026265 s.
Forward pass took 0.024098 s.
Forward pass took 0.022711 s.
Forward pass took 0.022912 s.
Forward pass took 0.0240732 s on average.
Converting model architecture.
Converting model weights.
Done converting model weights.
Calculating model hash.
Model conversion finished.
writing fdeep_model.json


In [0]:
!ls ./

fdeep_model.json  frugally-deep  keras_model.h5  sample_data


###Run with c++


1.   Download the json file
2.   Install some packages follwing the instruction: https://github.com/Dobiasd/frugally-deep/blob/master/INSTALL.md

        Requirement: cmake >= 3.2 and  c++ >= 14


3.   Copy and past the following c++ code
4.   Compile and Run it



// main.cpp


```
#include <fdeep/fdeep.hpp>
int main()
{
    const auto model = fdeep::load_model("fdeep_model.json");
    const auto result = model.predict(
        {fdeep::tensor(fdeep::tensor_shape(static_cast<std::size_t>(4)),
        {1, 2, 3, 4})});
    std::cout << fdeep::show_tensors(result) << std::endl;
}
```


